In [12]:
import advertools as adv
import pandas as pd
pd.options.display.max_columns = None

In [13]:
filename = 'filename'
domain = 'domain'
nyt_newssitemap = adv.sitemap_to_df('https://domain.com/sitemap.xml')

2022-01-01 10:19:59,248 | INFO | sitemaps.py:419 | sitemap_to_df | Getting https://www.aftership.com/couriers/sitemap-0.xml


In [14]:
adv.crawl(nyt_newssitemap['loc'][:100],
          output_file='%s/%s.jl' % (domain, filename))

2022-01-01 10:20:00 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-01-01 10:20:00 [scrapy.utils.log] INFO: Versions: lxml 4.7.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.10.1 (main, Dec  6 2021, 23:19:43) [Clang 12.0.0 (clang-1200.0.32.29)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1m  14 Dec 2021), cryptography 36.0.1, Platform macOS-10.15.7-x86_64-i386-64bit
2022-01-01 10:20:00 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-01-01 10:20:01 [scrapy.crawler] INFO: Overridden settings:
{'ROBOTSTXT_OBEY': True,
 'SPIDER_LOADER_WARN_ONLY': True,
 'USER_AGENT': 'advertools/0.12.3'}
2022-01-01 10:20:01 [scrapy.extensions.telnet] INFO: Telnet Password: 2b578a034a497116
2022-01-01 10:20:01 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.Fee

In [15]:
nyt_crawl_df = pd.read_json('%s/%s.jl' % (domain, filename), lines=True)

In [32]:
links = nyt_crawl_df['links_url'].str.split('@@').explode()

# Get links not containing your domain keywords
links = links[~links.str.contains(domain)]
links

1                         https://www.automizely.com/ko
1                        https://www.returnscenter.com/
1                              https://www.postmen.com/
1               https://www.automizely.com/ko/marketing
1            https://www.automizely.com/ko/dropshipping
                            ...                        
99    https://www.automizely.com/zh-Hant/company/cus...
99          https://www.automizely.com/zh-Hant/partners
99    https://www.automizely.com/zh-Hant/partners/re...
99        https://www.automizely.com/zh-Hant/affiliates
99             https://www.automizely.com/legal/cookies
Name: links_url, Length: 8316, dtype: object

In [33]:
adv.crawl(links, '%s/%s_check_links.jl' % (domain, filename)) 

2022-01-01 10:32:49 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-01-01 10:32:49 [scrapy.utils.log] INFO: Versions: lxml 4.7.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.10.1 (main, Dec  6 2021, 23:19:43) [Clang 12.0.0 (clang-1200.0.32.29)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1m  14 Dec 2021), cryptography 36.0.1, Platform macOS-10.15.7-x86_64-i386-64bit
2022-01-01 10:32:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-01-01 10:32:49 [scrapy.crawler] INFO: Overridden settings:
{'ROBOTSTXT_OBEY': True,
 'SPIDER_LOADER_WARN_ONLY': True,
 'USER_AGENT': 'advertools/0.12.3'}
2022-01-01 10:32:49 [scrapy.extensions.telnet] INFO: Telnet Password: 3fd4444b7b8e9303
2022-01-01 10:32:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.Fee

In [34]:
crawled_links = pd.read_json('%s/%s_check_links.jl' % (domain, filename), lines=True)

In [35]:
broken_links = crawled_links[crawled_links['status'] >= 400][['url', 'status']]
broken_links.to_csv('%s/%s_broken_links.csv' % (domain, filename))